In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [2]:
URL = "https://wuzzuf.net/search/jobs/?a=spbg&q=Data%20Analyst&start={}"

      #" https://wuzzuf.net/search/jobs/?a=hpb&q=Data%20Analyst&start={}"

response = requests.get(URL)

HEADERS = response.request.headers


In [3]:
def scrape_wuzzuf(num_pages=1):
    jobs = []

    for page in range(num_pages):
        print(f"Scraping page {page + 1}...")
        response = requests.get(URL.format(page), headers=HEADERS)

        if response.status_code != 200:
            print("Failed to retrieve data")
            continue

        soup = BeautifulSoup(response.text, "html.parser")

        # التأكد من أن الصفحة تحتوي على الوظائف
        job_cards = soup.find_all("div", class_="css-1gatmva")
        print(f"Found {len(job_cards)} jobs on page {page + 1}")

        for job in job_cards:
            try:
                all_spans = job.find_all("span")


                title = job.find("h2").get_text(strip=True) if job.find("h2") else "N/A"
                company = job.find("a", class_="css-17s97q8").get_text(strip=True) if job.find("a", class_="css-17s97q8") else "N/A"
                clocation = job.find("span", class_="css-5wys0k").get_text(strip=True) if job.find("span", class_="css-5wys0k") else "N/A"
                job_type = job.find("span", class_="css-1ve4b75 eoyjyou0").get_text(strip=True) if job.find("span", class_="css-1ve4b75 eoyjyou0") else "N/A"
                jlocation = job.find("span", class_="css-o1vzmt eoyjyou0").get_text(strip=True) if job.find("span", class_="css-o1vzmt eoyjyou0") else "N/A"





                #البحث عن سنوات الخبرة باستخدام regex داخل جميع الـ <span>
                yearsExps = "N/A"  # قيمة افتراضية في حالة عدم العثور على أي قيمة
                for span in all_spans:
                    match = re.search(r"(\d+\s*-\s*\d+)\s*Yrs\s*of\s*Exp", span.get_text(strip=True))
                    if match:
                        yearsExps = match.group(1)  # استخراج القيمة مثل "7 - 15"
                        break  # إذا وجدنا سنوات الخبرة، لا داعي للبحث في باقي الـ spans

                dur = job.find("div", class_="css-4c4ojb").get_text(strip=True) if job.find("div", class_="css-4c4ojb") else "N/A"
                link = "https://wuzzuf.net" + job.find("h2").find("a")["href"] if job.find("h2") and job.find("h2").find("a") else "N/A"

                jobs.append({
                    "Title": title,
                    "Company": company,
                    "Company Location": clocation,
                    "Job Location": jlocation,
                   # "Job Level": job_level,
                    "Experience Years": yearsExps,
                    "Duration of Job Title": dur,
                    "Job Type": job_type,
                    "Link": link
                })
            except AttributeError:
                continue

        time.sleep(2)  # انتظار لمنع الحظر

    return jobs

In [4]:
if __name__ == "__main__":
    num_pages = 15
    job_data = scrape_wuzzuf(num_pages)

    if job_data:
        df = pd.DataFrame(job_data)
        df.to_csv("wuzzuf_data_analyst_jobs.csv", index=False)
        print("Scraping completed. Data saved to wuzzuf_data_analyst_jobs.csv")
    else:
        print("No data scraped. Check your code.")


Scraping page 1...
Found 15 jobs on page 1
Scraping page 2...
Found 15 jobs on page 2
Scraping page 3...
Found 15 jobs on page 3
Scraping page 4...
Found 15 jobs on page 4
Scraping page 5...
Found 15 jobs on page 5
Scraping page 6...
Found 15 jobs on page 6
Scraping page 7...
Found 15 jobs on page 7
Scraping page 8...
Found 15 jobs on page 8
Scraping page 9...
Found 15 jobs on page 9
Scraping page 10...
Found 15 jobs on page 10
Scraping page 11...
Found 15 jobs on page 11
Scraping page 12...
Found 15 jobs on page 12
Scraping page 13...
Found 15 jobs on page 13
Scraping page 14...
Found 15 jobs on page 14
Scraping page 15...
Found 7 jobs on page 15
Scraping completed. Data saved to wuzzuf_data_analyst_jobs.csv


In [5]:
data = pd.read_csv("wuzzuf_data_analyst_jobs.csv")
data.head()

,Title,Company,Company Location,Job Location,Experience Years,Duration of Job Title,Job Type,Link
0,Data Analyst,Venus Hiring -,"Toronto,Canada",On-site,1 - 5,2 daysago,Full Time,https://wuzzuf.nethttps://wuzzuf.net/jobs/p/Ga...
1,Data Analyst Manager,WUZZUF -,"Maadi,Cairo,Egypt",Hybrid,7 - 15,20 hoursago,Full Time,https://wuzzuf.nethttps://wuzzuf.net/jobs/p/78...
2,Data Analyst,ECC Group -,"Nasr City,Cairo,Egypt",On-site,5 - 7,2 daysago,Full Time,https://wuzzuf.nethttps://wuzzuf.net/jobs/p/gg...
3,Junior Data Analyst,Confidential -,"Cairo,Egypt",On-site,2 - 4,5 daysago,Full Time,https://wuzzuf.nethttps://wuzzuf.net/jobs/p/kl...
4,Data Analyst,Luminar -,"New Cairo,Cairo,Egypt",Remote,2 - 5,NaN,Full Time,https://wuzzuf.nethttps://wuzzuf.net/jobs/p/cZ...
